# Workspace Manipulations

In this chapter, you will learn about various workspace manipulations including how to convert from HistFactory XML+ROOT workspaces to pyhf. We'll cover some common pitfalls such as locations of root files, and being able to set the base path for the conversion.

## Getting the XML+ROOT

Note, getting the XML+ROOT won't necessarily be covered as part of the tutorial.

If you want to practice extracting out the HistFactory files from the workspace, first create the workspace like so:

```
$ hist2workspace config/example.xml

... lots of output! ...
```

and you'll notice a few new files being made!

```
$ ls results/
total 136K
drwxr-xr-x 7 kratsg 224 Sep 23 16:32 ./
drwxr-xr-x 5 kratsg 160 Sep 23 16:33 ../
-rw-r--r-- 1 kratsg 503 Sep 23 16:32 example_GaussExample.root
-rw-r--r-- 1 kratsg 40K Sep 23 16:32 example_channel1_GaussExample_model.root
-rw-r--r-- 1 kratsg 38K Sep 23 16:32 example_channel2_GaussExample_model.root
-rw-r--r-- 1 kratsg 48K Sep 23 16:32 example_combined_GaussExample_model.root
-rw-r--r-- 1 kratsg  26 Sep 23 16:32 example_results.table
```

In particular, `example_combined_GaussExample_model.root` is exactly the file that contains the `RooStats::HistFactory::Measurement` object:

```
$ root results/example_combined_GaussExample_model.root 
   ------------------------------------------------------------
  | Welcome to ROOT 6.18/04                  https://root.cern |
  |                               (c) 1995-2019, The ROOT Team |
  | Built for macosx64 on Sep 11 2019, 15:38:23                |
  | From tags/v6-18-04@v6-18-04                                |
  | Try '.help', '.demo', '.license', '.credits', '.quit'/'.q' |
   ------------------------------------------------------------

root [0] 
Attaching file results/example_combined_GaussExample_model.root as _file0...

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

(TFile *) 0x7ffaa30d2130
root [1] .ls
TFile**		results/example_combined_GaussExample_model.root	
 TFile*		results/example_combined_GaussExample_model.root	
  KEY: RooWorkspace	combined;1	combined
  KEY: TProcessID	ProcessID0;1	e1e9272e-fddb-11ea-86b3-1556a8c0beef
  KEY: TDirectoryFile	channel1_hists;1	channel1_hists
  KEY: TDirectoryFile	channel2_hists;1	channel2_hists
  KEY: RooStats::HistFactory::Measurement	GaussExample;1
```

from which you can extract out the necessary XML files as well:

```
root [2] GaussExample->PrintXML()
Printing XML Files for measurement: GaussExample
Printing XML Files for channel: channel1
Finished printing XML files
Printing XML Files for channel: channel2
Finished printing XML files
Finished printing XML files
```

which dumps them into the same directory you ran from:

```
$ ls
total 12K
drwxr-xr-x 8 kratsg  256 Sep 23 16:32 ./
drwxr-xr-x 4 kratsg  128 Sep 22 15:10 ../
-rw-r--r-- 1 kratsg  459 Sep 23 16:32 GaussExample.xml
-rw-r--r-- 1 kratsg 1.1K Sep 23 16:32 GaussExample_channel1.xml
-rw-r--r-- 1 kratsg  795 Sep 23 16:32 GaussExample_channel2.xml
drwxr-xr-x 6 kratsg  192 Sep 22 15:10 config/
drwxr-xr-x 3 kratsg   96 Sep 22 15:10 data/
drwxr-xr-x 7 kratsg  224 Sep 23 16:32 results/
```

## XML to JSON

### via the command line

So pyhf comes with a lot of nifty utilities you can access. The documentation for the command line can be found via `pyhf --help` or [online](https://scikit-hep.org/pyhf/cli.html).

In [1]:
!pyhf --help

Usage: pyhf [OPTIONS] COMMAND [ARGS]...

  Top-level CLI entrypoint.

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cls          Compute CLs value(s) for a given pyhf workspace.
  combine      Combine two workspaces into a single workspace.
  completions  Generate shell completion code.
  digest       Use hashing algorithm to calculate the workspace digest.
  inspect      Inspect a pyhf JSON document.
  json2xml     Convert pyhf JSON back to XML + ROOT files.
  patchset     Operations involving patchsets.
  prune        Prune components from the workspace.
  rename       Rename components of the workspace.
  sort         Sort the workspace.
  xml2json     Entrypoint XML: The top-level XML file for the PDF...


Let's focus for now on `pyhf xml2json` which requires that you have installed `pyhf[xmlio]` (pyhf with the xmlio option).

```
python -m pip install pyhf[xmlio]
```

Again, the online documentation for this option is found [here](https://scikit-hep.org/pyhf/cli.html#pyhf-xml2json).

In [2]:
!pyhf xml2json --help

Usage: pyhf xml2json [OPTIONS] ENTRYPOINT_XML

  Entrypoint XML: The top-level XML file for the PDF definition.

Options:
  --basedir PATH                  The base directory for the XML files to
                                  point relative to.

  --output-file TEXT              The location of the output json file. If not
                                  specified, prints to screen.

  --track-progress / --hide-progress
  -h, --help                      Show this message and exit.


Let's remind ourselves of what the top-level XML file looks like, as this is the `ENTRYPOINT_XML`.

In [3]:
!tail -n +15 data/multichannel_histfactory/config/example.xml | cat -n

     1	<!DOCTYPE Combination  SYSTEM 'HistFactorySchema.dtd'>
     2	
     3	<Combination OutputFilePrefix="./results/example">
     4	  <Input>./config/example_signal.xml</Input>
     5	  <Input>./config/example_control.xml</Input>
     6	  <Measurement Name="GaussExample" Lumi="1." LumiRelErr="0.1" ExportOnly="True">
     7	    <POI>SigXsecOverSM</POI>
     8	    <ParamSetting Const="True">Lumi</ParamSetting>
     9	  </Measurement>
    10	</Combination>


So to explain these options:
* `basedir` specifies the base directory for where all the XML files are reference with respect to. As you can see from lines 3, 4, 5 - this should be the directory containing `results/` and `config/`
* `output-file` specifies the output JSON file. If one is not specified, this will print to the screen, which you can redirect into a file if you want (`pyhf xml2json ... > workspace.json`)
* `hide-progress` will disable showing the progress bars when running the script... but we like progress bars 🙂

Let's go ahead and run this command, but we won't specify the output file so it goes to the screen. We'll also disable the progress tracking, just so we have a nicer output for this tutorial.

In [ ]:
!pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | cat -n

Only 130 lines for the entire workspace! Not too shabby. If we look through a couple of pieces:
* line 2: specify a list of channels
* line 5: specify the samples for `channel1`
* lines 6-10: specify the expected event rate for the `signal` sample in `channel1`
* line 11: specify a list of modifiers (e.g. parameters that modify the sample)

Similarly, if we continue down to the second half of this JSON, we hit line 72 which specifies a list of `measurements` for this workspace. In fact, we only have one measurement called `GaussExample` with the parameter of interest defined as `SigXsecOverSM`. This measurement also specifies additional parameter configuration such as details for the luminosity modifier (parameter name `lumi`).

Nearly at the end, the next part of this specification is for the `observations` (observed data) on line 113. Each observation corresponds with the channel, where `channel1` has two bins, and `channel2` also has two bins.

Finally, we have a `version` which specifies the version of the schema used for the JSON HistFactory. In this case, we're using `1.0.0` which has the [https://scikit-hep.org/pyhf/schemas/1.0.0/workspace.json](https://scikit-hep.org/pyhf/schemas/1.0.0/workspace.json) definition which refers to the [https://scikit-hep.org/pyhf/schemas/1.0.0/defs.json](https://scikit-hep.org/pyhf/schemas/1.0.0/defs.json).

What's really nice about the schema definition is that it allows anyone to write their own tooling/scripting to build up the workspace and quickly check if it matches the schema. This will get you 90% of the way there in having a valid workspace to work with.

There are some additional checks that cannot be done, such as name conflicts, or ensuring that all samples in a channel have the same binning structure. The good news is that these checks can be done simply by loading up the workspace into a `pyhf.Workspace` object which will do the schema validation, as well as the additional checks.

Speaking of `pyhf.Workspace` objects...

### via the python interface

Let's do the exact same thing, but from the python interpreter

In [1]:
import pyhf
import pyhf.readxml # not imported by default!

spec = pyhf.readxml.parse('data/multichannel_histfactory/config/example.xml', 'data/multichannel_histfactory')

So we're not going to dump this out. We already did that above. Let's just quickly go ahead and load it into a [`pyhf.Workspace`](https://scikit-hep.org/pyhf/_generated/pyhf.workspace.Workspace.html#pyhf.workspace.Workspace) object because we can.

In [2]:
ws = pyhf.Workspace(spec)
print(f'    channels: {ws.channels}')
print(f'       nbins: {ws.channel_nbins}')
print(f'     samples: {ws.samples}')
print(f'   modifiers: {ws.modifiers}')
print(f'  parameters: {ws.parameters}')
print(f'observations: {ws.observations}')

    channels: ['channel1', 'channel2']
       nbins: {'channel1': 2, 'channel2': 2}
     samples: ['bkg', 'signal']
   modifiers: [('SigXsecOverSM', 'normfactor'), ('lumi', 'lumi'), ('uncorrshape_control', 'shapesys'), ('uncorrshape_signal', 'shapesys')]
  parameters: ['SigXsecOverSM', 'lumi', 'uncorrshape_control', 'uncorrshape_signal']
observations: {'channel1': [110.0, 155.0], 'channel2': [205.0, 345.0]}


Already, we're seeing a lot of information about this workspace as it's rather inspectable. Remember, this is not a model. What we call a 'model' is to combine the channel specification with a measurement... that is, a measurement of a workspace uniquely defines that model. A model might choose a particular parameter of interest to measure or set specific parameters as constant during the fit. These configurations are all stored in the `measurements` key we saw above. We'll explore more about models in the next chapter.

Let's move on to more things we can do with the command line.

## Workspace Inspection

Now that we have a working command for converting our XML to JSON, let's go ahead and take advantage of the JSON output by piping it to `pyhf inspect` which will print out a nice summary of our workspace.

In [3]:
! pyhf inspect --help

Usage: pyhf inspect [OPTIONS] [WORKSPACE]

  Inspect a pyhf JSON document.

  Example:

  .. code-block:: shell

      $ curl -sL https://raw.githubusercontent.com/scikit-
      hep/pyhf/master/docs/examples/json/2-bin_1-channel.json | pyhf inspect
      Summary         ------------------            channels  1
      samples  2          parameters  2           modifiers  2

             channels  nbins          ----------  -----       singlechannel
             2

              samples          ----------          background
              signal

           parameters  constraint              modifiers          ----------
           ----------              ----------                  mu
           unconstrained           normfactor     uncorr_bkguncrt
           constrained_by_poisson  shapesys

          measurement           poi            parameters
          ----------        ----------        ----------     (*) Measurement
          mu            (none)

Options:
  --output-file T

In [4]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf inspect

              Summary       
        ------------------  
           channels  2
            samples  2
         parameters  4
          modifiers  4

           channels  nbins
         ----------  -----
           channel1    2  
           channel2    2  

            samples
         ----------
                bkg
             signal

         parameters  constraint              modifiers
         ----------  ----------              ----------
      SigXsecOverSM  unconstrained           normfactor
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys
 uncorrshape_signal  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
   (*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



Immediately, we get a lot of useful information. We can see the number of channels, samples, parameters, and modifiers. Then we get a breakdown of the channels (and the number of bins for each channel), the samples, and the parameters. Finally, we see a list of measurements defined in the workspace, as well as the `(*)` denoting the default measurement if one is not specified.

**Could the number of parameters and modifiers differ?**

## "Normalizing" a Workspace

There comes a time when you need to make comparisons to determine changes between two workspaces. This means depending on how the workspace is generated, one might need to "sort" it. `pyhf sort` is a utility that will normalize the workspace for you, such that certain operations like calculating a checksum (`pyhf digest`) guarantees unitarity.

For simple workspaces like the ones we're using in this tutorial, they're already sorted... however, this is not true in the real world. Notice how the `bkg` is now the first sample and `signal` is the second sample after sorting.

In [5]:
! pyhf sort --help

Usage: pyhf sort [OPTIONS] [WORKSPACE]

  Sort the workspace.

  See :func:`pyhf.workspace.Workspace.sorted` for more information.

  Example:

  .. code-block:: shell

      $ curl -sL https://raw.githubusercontent.com/scikit-
      hep/pyhf/master/docs/examples/json/2-bin_1-channel.json | pyhf sort |
      jq '.' | md5     8be5186ec249d2704e14dd29ef05ffb0

  .. code-block:: shell

      $ curl -sL https://raw.githubusercontent.com/scikit-
      hep/pyhf/master/docs/examples/json/2-bin_1-channel.json | jq -S '.chan
      nels|=sort_by(.name)|.channels[].samples|=sort_by(.name)|.channels[].s
      amples[].modifiers|=sort_by(.name,.type)|.observations|=sort_by(.name)
      ' | md5     8be5186ec249d2704e14dd29ef05ffb0

Options:
  --output-file TEXT  The location of the output json file. If not specified,
                      prints to screen.

  -h, --help          Show this message and exit.


In [6]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf sort

{
    "channels": [
        {
            "name": "channel1",
            "samples": [
                {
                    "data": [
                        100.0,
                        150.0
                    ],
                    "modifiers": [
                        {
                            "data": null,
                            "name": "lumi",
                            "type": "lumi"
                        },
                        {
                            "data": [
                                10.000000149011612,
                                10.000000521540642
                            ],
                            "name": "uncorrshape_signal",
                            "type": "shapesys"
                        }
                    ],
                    "name": "bkg"
                },
                {
                    "data": [
                        10.0,
                        35.0
                    ],
                    "modifier

## Computing a digest

Next up is a way to determine if two workspaces are equivalent, simply by comparing their computed digest. Note that this is based on the contents of the workspace and will not ensure floating-point differences are treated identically. That is, `2.19999999` and `2.2000001` will likely be treated as differently in the digest calculation as in python. We'll show here why sorting is very important.

In [7]:
! pyhf digest --help

Usage: pyhf digest [OPTIONS] [WORKSPACE]

  Use hashing algorithm to calculate the workspace digest.

  Returns:     digests (:obj:`dict`): A mapping of the hashing algorithms
  used to the computed digest for the workspace.

  Example:

  .. code-block:: shell

      $ curl -sL https://raw.githubusercontent.com/scikit-
      hep/pyhf/master/docs/examples/json/2-bin_1-channel.json | pyhf digest
      sha256:dad8822af55205d60152cbe4303929042dbd9d4839012e055e7c6b6459d68d7
      3

Options:
  -a, --algorithm TEXT          The hashing algorithm used to compute the
                                workspace digest.

  -j, --json / -p, --plaintext  Output the hash values as a JSON dictionary or
                                plaintext strings

  -h, --help                    Show this message and exit.


In [8]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf digest

sha256:50165e8ef034c514fb77e8f05a15a002c02bd659f001657952b79e0552470f79


In [9]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf sort | \
  pyhf digest

sha256:27a35f6874cf91f9b38916cf948ac18ee650f1b578a93107b9b212c8752b1310


Remember that the ordering of the samples will have switched through the sorting.

The `sha256` algorithm is used to compute the checksum for this workspace. This means that one can generally "normalize" all workspaces, then compute the digest and guarantee uniqueness. As with all command line functionality you've seen so far, there are equivalent ways to do it through python.

In [10]:
print(f'Unsorted: {pyhf.utils.digest(ws)}')
print(f'Sorted:   {pyhf.utils.digest(pyhf.Workspace.sorted(ws))}')

Unsorted: 50165e8ef034c514fb77e8f05a15a002c02bd659f001657952b79e0552470f79
Sorted:   27a35f6874cf91f9b38916cf948ac18ee650f1b578a93107b9b212c8752b1310


## "Pruning" away items

Sometimes you want to manipulate workspaces by removing channels or samples or systematics (or measurements). This can be useful when trying to debug fits, or to build background-only workspaces, or to clean up a workspace.

In [11]:
!pyhf prune --help

Usage: pyhf prune [OPTIONS] [WORKSPACE]

  Prune components from the workspace.

  See :func:`pyhf.workspace.Workspace.prune` for more information.

Options:
  --output-file TEXT              The location of the output json file. If not
                                  specified, prints to screen.

  -c, --channel <CHANNEL>...
  -s, --sample <SAMPLE>...
  -m, --modifier <MODIFIER>...
  -t, --modifier-type [histosys|lumi|normfactor|normsys|shapefactor|shapesys|staterror]
  --measurement <MEASUREMENT>...
  -h, --help                      Show this message and exit.


### prune channels

In [12]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf prune -c channel1 | \
  pyhf inspect

              Summary       
        ------------------  
           channels  1
            samples  1
         parameters  2
          modifiers  2

           channels  nbins
         ----------  -----
           channel2    2  

            samples
         ----------
                bkg

         parameters  constraint              modifiers
         ----------  ----------              ----------
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
   (*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



### prune samples

In [13]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf prune -s signal | \
  pyhf inspect

              Summary       
        ------------------  
           channels  2
            samples  1
         parameters  3
          modifiers  3

           channels  nbins
         ----------  -----
           channel1    2  
           channel2    2  

            samples
         ----------
                bkg

         parameters  constraint              modifiers
         ----------  ----------              ----------
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys
 uncorrshape_signal  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
   (*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



### prune modifiers

In [14]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf prune -m uncorrshape_signal | \
  pyhf inspect

              Summary       
        ------------------  
           channels  2
            samples  2
         parameters  3
          modifiers  3

           channels  nbins
         ----------  -----
           channel1    2  
           channel2    2  

            samples
         ----------
                bkg
             signal

         parameters  constraint              modifiers
         ----------  ----------              ----------
      SigXsecOverSM  unconstrained           normfactor
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
   (*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



### prune modifier types

In [15]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf prune -t shapesys | \
  pyhf inspect

        Summary       
  ------------------  
     channels  2
      samples  2
   parameters  2
    modifiers  2

     channels  nbins
   ----------  -----
     channel1    2  
     channel2    2  

      samples
   ----------
          bkg
       signal

   parameters  constraint              modifiers
   ----------  ----------              ----------
SigXsecOverSM  unconstrained           normfactor
         lumi  constrained_by_normal   lumi

  measurement           poi            parameters
   ----------        ----------        ----------
(*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



## Renaming items

In addition to removing items, you might want to rename your channels, samples, modifiers, or measurement names. This can be useful for creating modifier correlations, or removing modifier correlations, or just cleaning up your workspace to get it ready for publication.

In [16]:
! pyhf rename --help

Usage: pyhf rename [OPTIONS] [WORKSPACE]

  Rename components of the workspace.

  See :func:`pyhf.workspace.Workspace.rename` for more information.

Options:
  --output-file TEXT              The location of the output json file. If not
                                  specified, prints to screen.

  -c, --channel <PATTERN> <REPLACE>...
  -s, --sample <PATTERN> <REPLACE>...
  -m, --modifier <PATTERN> <REPLACE>...
  --measurement <PATTERN> <REPLACE>...
  -h, --help                      Show this message and exit.


### rename channels

In [17]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf rename -c channel1 SR -c channel2 CR | \
  pyhf inspect

              Summary       
        ------------------  
           channels  2
            samples  2
         parameters  4
          modifiers  4

           channels  nbins
         ----------  -----
                 CR    2  
                 SR    2  

            samples
         ----------
                bkg
             signal

         parameters  constraint              modifiers
         ----------  ----------              ----------
      SigXsecOverSM  unconstrained           normfactor
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys
 uncorrshape_signal  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
   (*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



### rename samples

In [18]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf rename -s bkg background | \
  pyhf inspect

              Summary       
        ------------------  
           channels  2
            samples  2
         parameters  4
          modifiers  4

           channels  nbins
         ----------  -----
           channel1    2  
           channel2    2  

            samples
         ----------
         background
             signal

         parameters  constraint              modifiers
         ----------  ----------              ----------
      SigXsecOverSM  unconstrained           normfactor
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys
 uncorrshape_signal  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
   (*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



### rename modifiers

In [19]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf rename -m uncorrshape_signal corrshape -m uncorrshape_control corrshape | \
  pyhf inspect

        Summary       
  ------------------  
     channels  2
      samples  2
   parameters  3
    modifiers  3

     channels  nbins
   ----------  -----
     channel1    2  
     channel2    2  

      samples
   ----------
          bkg
       signal

   parameters  constraint              modifiers
   ----------  ----------              ----------
SigXsecOverSM  unconstrained           normfactor
    corrshape  constrained_by_poisson  shapesys
         lumi  constrained_by_normal   lumi

  measurement           poi            parameters
   ----------        ----------        ----------
(*) GaussExample      SigXsecOverSM       lumi,SigXsecOverSM



### rename measurements

In [20]:
! pyhf xml2json --basedir data/multichannel_histfactory data/multichannel_histfactory/config/example.xml --hide-progress | \
  pyhf rename --measurement GaussExample FitConfig | \
  pyhf inspect

              Summary       
        ------------------  
           channels  2
            samples  2
         parameters  4
          modifiers  4

           channels  nbins
         ----------  -----
           channel1    2  
           channel2    2  

            samples
         ----------
                bkg
             signal

         parameters  constraint              modifiers
         ----------  ----------              ----------
      SigXsecOverSM  unconstrained           normfactor
               lumi  constrained_by_normal   lumi
uncorrshape_control  constrained_by_poisson  shapesys
 uncorrshape_signal  constrained_by_poisson  shapesys

        measurement           poi            parameters
         ----------        ----------        ----------
      (*) FitConfig      SigXsecOverSM       lumi,SigXsecOverSM

